In [1]:
from pgtools import gff_parser
import os
import pandas as pd

In [2]:
gffs_dir = "/home/pampuch/studia/magisterka/final_summaries/gffs/"
panaroo_cores_dir = "/home/pampuch/studia/magisterka/final_summaries/core_summaries_new/panaroo_model/"

In [4]:
def get_cds_sum_lens(gff):
    cds_lens = {}
    gff = gff_parser.parse_joined_gff(gff)
    for cds in gff:
        cds_len = cds.end - cds.start + 1
        if cds.seq_name in cds_lens:
            if cds.annotation_id in cds_lens[cds.seq_name]:
                cds_lens[cds.seq_name][cds.annotation_id] += cds_len
            else:
                cds_lens[cds.seq_name][cds.annotation_id] = cds_len
        else:
            cds_lens[cds.seq_name] = {cds.annotation_id : cds_len}
    return cds_lens

In [5]:
dataset="GPSC_subset"
gpsc_gff = os.path.join(gffs_dir, "cds_inter_cact", dataset, "cds_inter_cact.gff")

In [6]:
lens_gpsc = get_cds_sum_lens(gpsc_gff)

In [7]:
sum_gpsc = 0

for k, vals in lens_gpsc.items():
    for annot, ann_len in vals.items():
        sum_gpsc += ann_len

In [10]:
cactus_sum_cds_lens = {}
for dataset in os.listdir(panaroo_cores_dir):
    print(dataset)
    sum_lens = 0
    lens_all = get_cds_sum_lens(os.path.join(gffs_dir, "cds_inter_cact", dataset, "cds_inter_cact.gff"))
    for k, vals in lens_all.items():
        for annot, ann_len in vals.items():
            sum_lens += ann_len
    cactus_sum_cds_lens[dataset] = sum_lens

sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1_fragmented
sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1_contaminated
klebsiella_subset
sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1
GPSC_subset


In [11]:
cactus_sum_cds_lens

{'sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1_fragmented': 115492867,
 'sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1_contaminated': 270713583,
 'klebsiella_subset': 21064264,
 'sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1': 274189200,
 'GPSC_subset': 14121688}

In [12]:
cactus_sum_all_lens = {}

In [ ]:
cont_lens_dir = "/home/pampuch/studia/magisterka/final_summaries/scaf_cover/contig_lens_in_model/"
scaff_lens = "/home/pampuch/studia/magisterka/final_summaries/scaffols_lens"
cont_csv_name = "scaff_cont_lens.csv"
all_lens_csv = "scaf_lens.csv"

gffs_dir = "/home/pampuch/studia/magisterka/final_summaries/gffs/"
# panaroo_cores_dir = "/home/pampuch/studia/magisterka/final_summaries/core_summaries_new/panaroo_model/"

In [ ]:
def summarise_dataset_coverages(dataset):
    for model in os.listdir(cont_lens_dir):
        print(model)
        df_cont_lens = pd.read_csv(os.path.join(cont_lens_dir, model, dataset, cont_csv_name))

            
        df_lens = pd.read_csv(os.path.join(scaff_lens, dataset, all_lens_csv))
        
        if dataset == "GPSC_subset" and model != "panaroo":       
            df_lens = pd.read_csv(os.path.join(scaff_lens, dataset, "scaf_lens_without_velvet.csv"))
        # print(df_lens.head())
        # print(df_cont_lens.head())
        df_cov = df_lens.join(df_cont_lens.set_index("Contig"), on="Contig")

### Chcę otrzymać:
1. pokrycie modelu przez CDS
2. pokrycie CDS przez model
3. pokrycie genomów przez model

In [44]:
cont_lens_dir = "/home/pampuch/studia/magisterka/final_summaries/scaf_cover/contig_lens_in_model/"
scaff_lens = "/home/pampuch/studia/magisterka/final_summaries/scaffols_lens"
cont_csv_name = "scaff_cont_lens.csv"
all_lens_csv = "scaf_lens.csv"
def model_cds_cov(model, dataset):
    summ_dict = {"Pokrycie pangenomu": None, "Pokrycie modelu przez CDS": None, "Pokrycie CDS przez model": None}
    print(dataset)
    df_cont_lens = pd.read_csv(os.path.join(cont_lens_dir, model, dataset, cont_csv_name))
        
    df_lens = pd.read_csv(os.path.join(scaff_lens, dataset, all_lens_csv))
    
    if dataset == "GPSC_subset" and model != "panaroo":       
        df_lens = pd.read_csv(os.path.join(scaff_lens, dataset, "scaf_lens_without_velvet.csv"))
    df_cov = df_lens.join(df_cont_lens.set_index("Contig"), on="Contig")
    # return df_cov
    genome_cvg = df_cov["sum of seq lens in model"].sum() / df_cov["Len"].sum()
    summ_dict["Pokrycie pangenomu"] = genome_cvg
    model_len = df_cov["sum of seq lens in model"].sum()

    if model == "cactus":
        gffs_in_model = os.path.join(gffs_dir, "cds_inter_cact", dataset, "/studia/magisterka/final_summaries/gffs/prog_core")

    if model == "progressive":
        gffs_in_model = os.path.join("/home/pampuch/studia/magisterka/final_summaries/gffs/prog_core/cds_inter_prog/", dataset, "cds_inter_prog.gff")
        
    sum_lens = 0
    lens_all = get_cds_sum_lens(gffs_in_model)
    for k, vals in lens_all.items():
        for annot, ann_len in vals.items():
            sum_lens += ann_len
            
    cds_in_model_len = sum_lens
    summ_dict["Pokrycie modelu przez CDS"] = cds_in_model_len *100 / model_len

    
    lens_all = get_cds_sum_lens(os.path.join(gffs_dir, "cds_gffs", dataset, "all.gff"))
    sum_lens = 0
    # lens_all = get_cds_sum_lens(all_gffs)
    for k, vals in lens_all.items():
        for annot, ann_len in vals.items():
            sum_lens += ann_len
    cds_sum_lens = sum_lens
    summ_dict["Pokrycie CDS przez model"] = cds_in_model_len*100 / cds_sum_lens
    return summ_dict

In [42]:
datset="GPSC_subset"
model_cds_cov("cactus", dataset)

FileNotFoundError: [Errno 2] No such file or directory: '/studia/magisterka/final_summaries/gffs/prog_core'

In [45]:
datset="GPSC_subset"
model_cds_cov("progressive", dataset)

GPSC_subset


{'Pokrycie pangenomu': 0.8508167573461013,
 'Pokrycie modelu przez CDS': 87.1991247628826,
 'Pokrycie CDS przez model': 87.16098417577135}

In [48]:
dataset="sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1_fragmented"
model_cds_cov("progressive", dataset)

sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1_fragmented


{'Pokrycie pangenomu': 0.9652962654020067,
 'Pokrycie modelu przez CDS': 81.91096325099917,
 'Pokrycie CDS przez model': 97.14066807832825}